In [74]:
import random
import time
import redis
# 與LongDB連線
import pyodbc
r = redis.Redis(host='192.168.80.166', port=6379)

In [96]:
# function from other team member
# 使用方程式取得不同運動類型及地區的賽事資料
def game(type2,place):
    # 篩選賽事類型
    type2_dict = {"ride":" '自行車' ","run":" '路跑' "}
    x = type2_dict.get(type2)
    res_list = []
    # 篩選賽事區域
    if place == 'North':
        j = cursor.execute("SELECT * FROM SPLICE.RACE_BASIC WHERE 賽事種類 LIKE"+ x +"AND 日期 >'2019-01-01'  INTERSECT SELECT * FROM SPLICE.RACE_BASIC WHERE 地點 LIKE '%台北%' OR 地點 LIKE '%新北%' OR 地點 LIKE '%基隆%' OR 地點 LIKE '%桃園%' OR 地點 LIKE '%新竹%' OR 地點 LIKE '%宜蘭%'")
    elif place == 'Center':
        j = cursor.execute("SELECT * FROM SPLICE.RACE_BASIC WHERE 賽事種類 LIKE"+ x +"AND 日期 >'2019-01-01'  INTERSECT SELECT * FROM SPLICE.RACE_BASIC WHERE 地點 LIKE '%苗栗%' OR 地點 LIKE '%台中%' OR 地點 LIKE '%彰化%' OR 地點 LIKE '%南投%' OR 地點 LIKE '%雲林%'")
    elif place == 'South':
        j = cursor.execute("SELECT * FROM SPLICE.RACE_BASIC WHERE 賽事種類 LIKE"+ x +"AND 日期 >'2019-01-01'  INTERSECT SELECT * FROM SPLICE.RACE_BASIC WHERE 地點 LIKE '%嘉義%' OR 地點 LIKE '%台南%' OR 地點 LIKE '%高雄%' OR 地點 LIKE '%屏東%' OR 地點 LIKE '%澎湖%'")
    elif place == 'East':
        j = cursor.execute("SELECT * FROM SPLICE.RACE_BASIC WHERE 賽事種類 LIKE"+ x +"AND 日期 >'2019-01-01'  INTERSECT SELECT * FROM SPLICE.RACE_BASIC WHERE 地點 LIKE '%花蓮%' OR 地點 LIKE '%台東%'")
    # 建立字典清單
    for i in j:
        type4 = i[2]
        name = i[3]
        url = i[4]
        res = {'type': type4, 'name': name, 'url':url}
        res_list.append(res)
    return res_list

In [98]:
# 使用迴圈自動輸入到redis
location = ["North", "Center", "South", "East"]
while True:
    try:
        # 每次隨機運動地區各3筆賽事資料list初始化
        running_location = []
        cycling_location = []
        # 與longDB連線
        cnxn=pyodbc.connect("DSN=SpliceODBC64")
        # SQL指令容器
        cursor=cnxn.cursor()
        for place in location:
            # 使用random.sample取得函式結果為list中任意k=3個（該地區3筆跑步賽事資料）
            running_list = random.sample(game("run", place), k=3)
            # 將取得的賽事資料以"|"切割方便前端取用時分割，再加入list中便可用index分辨不同地區
            running_location.append(str(running_list[0])+"|"+str(running_list[1])+"|"+str(running_list[2]))

            cycling_list = random.sample(game("ride", place), k=3)
            cycling_location.append(str(cycling_list[0])+"|"+str(cycling_list[1])+"|"+str(cycling_list[2]))
        # 關閉與LongDB連線
        cursor.close()
        cnxn.close()
        
        # 將地區為key，三筆賽事資料為value輸入到redis
        r.hmset('running_race', {location[0]: running_location[0], location[1]: running_location[1],
                                      location[2]: running_location[2], location[3]: running_location[3]})
        r.hmset('cycling_race', {location[0]: cycling_location[0], location[1]: cycling_location[1],
                                      location[2]: cycling_location[2], location[3]: cycling_location[3]})
        
        # 測試是否取得北部地區隨機3筆賽事資料
        print(r.hmget('running_race', ["North"])[0].decode())
        print('-----------------------------------------------------')
        time.sleep(1800)
    except KeyboardInterrupt:
        break

{'type': '路跑', 'name': '2020 渣打臺北公益馬拉松', 'url': 'http://scbmarathon.com/'}|{'type': '路跑', 'name': '最強市民飆5K－新北市5000公尺挑戰賽', 'url': 'http://twrunner.soonnet.org:8012/road_run/index.html'}|{'type': '路跑', 'name': '2019 世界地球日路跑', 'url': 'https://www.eventpal.com.tw/FOAS/actions/ActivityIndex.action?showTabContent&seqno=8ebddf08-b21e-469b-9180-b0a57f9ec137'}
-----------------------------------------------------


KeyboardInterrupt: 

In [ ]:
# while True:
#     try:
#         # 與longDB連線
#         cnxn=pyodbc.connect("DSN=SpliceODBC64")
#         # SQL指令容器
#         cursor=cnxn.cursor()
#         running_north_list = random.sample(game("run","north"), k=3)
#         running_north = str(running_north_list[0])+"|"+str(running_north_list[1])+"|"+str(running_north_list[2])
#         print(running_north)
#         print('-----------------------------------------------------')
#         running_center_list = random.sample(game("run","center"), k=3)
#         running_center = str(running_center_list[0])+"|"+str(running_center_list[1])+"|"+str(running_center_list[2])
#         print(running_center)
#         print('-----------------------------------------------------')
#         running_south_list = random.sample(game("run","south"), k=3)
#         running_south = str(running_south_list[0])+"|"+str(running_south_list[1])+"|"+str(running_south_list[2])
#         print(running_south)
#         print('-----------------------------------------------------')
#         running_east_list = random.sample(game("run","east"), k=3)
#         running_east = str(running_east_list[0])+"|"+str(running_east_list[1])+"|"+str(running_east_list[2])
#         print(running_east)
#         print('-----------------------------------------------------')
#         cycling_north_list = random.sample(game("ride","north"), k=3)
#         cycling_north = str(cycling_north_list[0])+"|"+str(cycling_north_list[1])+"|"+str(cycling_north_list[2])
#         print(cycling_north)
#         print('-----------------------------------------------------')
#         cycling_center_list = random.sample(game("ride","center"), k=3)
#         cycling_center = str(cycling_center_list[0])+"|"+str(cycling_center_list[1])+"|"+str(cycling_center_list[2])
#         print(cycling_center)
#         print('-----------------------------------------------------')
#         cycling_south_list = random.sample(game("ride","south"), k=3)
#         cycling_south = str(cycling_south_list[0])+"|"+str(cycling_south_list[1])+"|"+str(cycling_south_list[2])
#         print(cycling_south)
#         print('-----------------------------------------------------')
#         cycling_east_list = random.sample(game("ride","east"), k=3)
#         cycling_east = str(cycling_east_list[0])+"|"+str(cycling_east_list[1])+"|"+str(cycling_east_list[2])
#         print(cycling_east)
#         print('-----------------------------------------------------')
#         cursor.close()
#         cnxn.close()
        
#         r.hmset('running_race', {'North': running_north, 'Center': running_center, 'South': running_south, 'East': running_east})
#         r.hmset('cycling_race', {'North': cycling_north, 'Center': cycling_center, 'South': cycling_south, 'East': cycling_east})

#         time.sleep(3600)
#     except KeyboardInterrupt:
#         break

In [4]:
# R = r.hmget('running_race',['North', 'Center', 'South', 'East']) #R是跑步list
# print(R[0].decode()) #北
# print(R[1].decode()) #中
# print(R[2].decode()) #南
# print(R[3].decode()) #東
# C = r.hmget('cycling_race',['North', 'Center', 'South', 'East']) #C是單車list